# Imports

In [1]:
import math
import torch
import torch.nn as nn
import torchtext
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset

# Dataset Loading

In [2]:
dataset = load_dataset("glue", "sst2")
train_dataset = dataset["train"]
test_dataset = dataset["validation"]

df_train = pd.DataFrame(train_dataset)
df_test = pd.DataFrame(test_dataset)
df_train.set_index('idx', inplace=True)
df_test.set_index('idx', inplace=True)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Build Vocabulary

In [3]:
# Load tokenizer
tokenizer = get_tokenizer('basic_english')

text = 'this is text'
print(tokenizer(text))

['this', 'is', 'text']


In [5]:
# Initialize training data iterator
class TextIter(torch.utils.data.Dataset):

  def __init__(self, input_data):
      self.text = input_data['sentence'].values.tolist()
  def __len__(self):
      return len(self.text)
  def __getitem__(self, idx):
      return self.text[idx]

# Build vocabulary
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

data_iter = TextIter(df_train)
vocab = build_vocab_from_iterator(yield_tokens(data_iter), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])
print(vocab.get_stoi())

{'zoom': 14761, 'zigzag': 14760, 'zeus': 14759, 'zealously': 14758, 'zaza': 14757, 'yung': 14756, 'xtc': 14754, 'winged': 14752, 'whitaker': 14751, 'wes': 14749, 'wedge': 14748, 'ward': 14746, 'visualizing': 14744, 'victor': 14743, 'veggietales': 14740, 'uzumaki': 14738, 'uwe': 14737, 'unseen': 14735, 'unrequited': 14734, 'unrepentant': 14733, 'unconned': 14728, 'um': 14727, 'ub': 14726, 'two-bit': 14723, 'trimming': 14719, 'tightly': 14712, 'thi': 14706, 'theorizing': 14705, 'talking-animal': 14703, 'swirl': 14698, 'sweep': 14697, 'surviving': 14696, 'superheroics': 14694, 'sunday': 14693, 'summary': 14692, 'strongman': 14688, 'stomp': 14685, 'steadfastly': 14681, 'star/producer': 14678, 'stalked': 14677, 'staggering': 14676, 'spotlights': 14674, 'spiderman': 14673, 'spaniel-eyed': 14670, 'spaceship': 14668, 'snowman': 14663, 'snoots': 14662, 'slc': 14660, 'shut': 14654, 'shunji': 14653, 'shrapnel': 14652, 'shellshock': 14650, 'shankman': 14649, 'seriocomic': 14647, 'seigner': 14646, 

In [6]:
text_unk = 'this is jkjkj' # jkjkj is an unknown vocab
seq_unk = [vocab[word] for word in tokenizer(text_unk)]

print(tokenizer(text_unk))
print(seq_unk)

['this', 'is', 'jkjkj']
[22, 11, 1]


In [7]:
# We will use this example throughout the article
text = 'this is text' 
seq = [vocab[word] for word in tokenizer(text)]

print(tokenizer(text))
print(seq)

['this', 'is', 'text']
[22, 11, 2302]


# Word Embedding

In [8]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab_size):
        super(Embeddings, self).__init__()
        self.emb = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.emb(x) * math.sqrt(self.d_model)

In [9]:
hidden_size = 4

input_data = torch.LongTensor(seq).unsqueeze(0)
emb_model = Embeddings(hidden_size, len(vocab))
token_emb = emb_model(input_data) 
print(f'Size of token embedding: {token_emb.size()}')

Size of token embedding: torch.Size([1, 3, 4])


# Positional Encoding

In [10]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
  
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [11]:
pe_model = PositionalEncoding(d_model=4, vocab_size=len(vocab))
output_pe = pe_model(token_emb)

print(f'Size of output embedding: {output_pe.size()}')

Size of output embedding: torch.Size([1, 3, 4])


# Self-Attention

In [12]:
class SingleHeadAttention(nn.Module):
    def __init__(self, d_model, d_head_size):
        super().__init__()
        self.lin_key = nn.Linear(d_model, d_head_size, bias=False)
        self.lin_query = nn.Linear(d_model, d_head_size, bias=False)
        self.lin_value = nn.Linear(d_model, d_head_size, bias=False)
        self.d_model = d_model

    def forward(self, x):
        query = self.lin_query(x)
        key = self.lin_key(x)
        value = self.lin_value(x)

        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.d_model)
        p_attn = scores.softmax(dim=-1)
        x = torch.matmul(p_attn, value)

        return x

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0
        d_k = d_model // h
        self.multi_head = nn.ModuleList([SingleHeadAttention(d_model, d_k) for _ in range(h)])
        self.lin_agg = nn.Linear(d_model, d_model)

    def forward(self, x):
        x = torch.cat([head(x) for head in self.multi_head], dim=-1)
        return self.lin_agg(x)

In [14]:
mult_att = MultiHeadAttention(h=2, d_model=4)
output_mult_att = mult_att(output_pe)

print(f'Size of output embedding after multi-head attention: {output_mult_att.size()}')

Size of output embedding after multi-head attention: torch.Size([1, 3, 4])


# Residual Connection

In [15]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(d_model))
        self.b_2 = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class ResidualConnection(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()
        self.norm = LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x1, x2):
        return self.dropout(self.norm(x1 + x2))

In [16]:
res_conn_1 = ResidualConnection(d_model=4)
output_res_conn_1 = res_conn_1(output_pe, output_mult_att)

print(f'Size of output embedding after residual connection: {output_res_conn_1.size()}')

Size of output embedding after residual connection: torch.Size([1, 3, 4])


# Feed-Forward

In [17]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [18]:
ff = FeedForward(d_model=4, d_ff=12)
output_ff = ff(output_res_conn_1)

print(f'Size of output embedding after feed-forward network: {output_ff.size()}')

Size of output embedding after feed-forward network: torch.Size([1, 3, 4])


In [19]:
res_conn_2 = ResidualConnection(d_model=4)
output_res_conn_2 = res_conn_2(output_res_conn_1, output_ff)

print(f'Size of output embedding after second residual: {output_res_conn_2.size()}')

Size of output embedding after second residual: torch.Size([1, 3, 4])


# Encoder Stack

In [20]:
class SingleEncoder(nn.Module):
    def __init__(self, d_model, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.res_1 = ResidualConnection(d_model, dropout)
        self.res_2 = ResidualConnection(d_model, dropout)

        self.d_model = d_model

    def forward(self, x):
        x_attn = self.self_attn(x)
        x_res_1 = self.res_1(x, x_attn)
        x_ff = self.feed_forward(x_res_1)
        x_res_2 = self.res_2(x_res_1, x_ff)

        return x_res_2

In [21]:
class EncoderBlocks(nn.Module):
    def __init__(self, layer, N):
        super().__init__()
        self.layers = nn.ModuleList([layer for _ in range(N)])
        self.norm = LayerNorm(layer.d_model)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.norm(x)

# Transformer Encoder Model

### Number of Layers, No. of heads, params are present here

In [22]:
class TransformerEncoderModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, d_ff, N,
                dropout=0.1):
        super().__init__()
        assert d_model % nhead == 0, "nheads must divide evenly into d_model"

        self.emb = Embeddings(d_model, vocab_size)
        self.pos_encoder = PositionalEncoding(d_model=d_model, vocab_size=vocab_size)

        attn = MultiHeadAttention(nhead, d_model)
        ff = FeedForward(d_model, d_ff, dropout)
        self.transformer_encoder = EncoderBlocks(SingleEncoder(d_model, attn, ff, dropout), N)
        self.classifier = nn.Linear(d_model, 2)
        self.d_model = d_model

    def forward(self, x):
        x = self.emb(x) * math.sqrt(self.d_model)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerEncoderModel(len(vocab), d_model=300, nhead=4, d_ff=50, 
                                    N=6, dropout=0.1).to(device)

/data/BADRI/ACAD/OML/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


# Dataloader

In [23]:
class TextDataset(torch.utils.data.Dataset):

  def __init__(self, input_data):        
      self.text = input_data['sentence'].values.tolist()
      self.label = input_data['label'].values.tolist()

  def __len__(self):
      return len(self.label)
    
  def get_sequence_token(self, idx):
      sequence = [vocab[word] for word in tokenizer(self.text[idx])]
      len_seq = len(sequence)
      return sequence, len_seq
  
  def get_labels(self, idx):
      return self.label[idx]

  def __getitem__(self, idx):
      sequence, len_seq = self.get_sequence_token(idx)
      label = self.get_labels(idx)
      return sequence, label, len_seq

def collate_fn(batch):
  
    sequences, labels, lengths = zip(*batch)
    max_len = max(lengths)
    
    for i in range(len(batch)):
        if len(sequences[i]) != max_len:
          for j in range(len(sequences[i]),max_len):
            sequences[i].append(0)

    return torch.tensor(sequences, dtype=torch.long), torch.tensor(labels, dtype=torch.long)

# Model Training

In [ ]:
def train(model, dataset, epochs, lr, bs):

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam((p for p in model.parameters() 
      if p.requires_grad), lr=lr)
    train_dataset = TextDataset(dataset)
    train_dataloader = DataLoader(train_dataset, num_workers=1, batch_size=bs, collate_fn=collate_fn, shuffle=True)
    
    # Training loop
    for epoch in range(epochs):
        total_loss_train = 0
        total_acc_train = 0   
        for train_sequence, train_label in tqdm(train_dataloader):
            
            # Model prediction
            predictions = model(train_sequence.to(device))
            labels = train_label.to(device)
            loss = criterion(predictions, labels)

            # Calculate accuracy and loss per batch
            correct = predictions.argmax(axis=1) == labels
            acc = correct.sum().item() / correct.size(0)
            total_acc_train += correct.sum().item()
            total_loss_train += loss.item()

            # Backprop
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()

        print(f'Epochs: {epoch + 1} | Loss: {total_loss_train / len(train_dataset): .3f} | Accuracy: {total_acc_train / len(train_dataset): .3f}')
    
epochs = 15
lr = 1e-4
batch_size = 4
train(model, df_train, epochs, lr, batch_size)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 16838/16838 [09:08<00:00, 30.71it/s]


Epochs: 1 | Loss:  0.158 | Accuracy:  0.656


100%|██████████████████████████████████████████████████████████████████████████████████████████| 16838/16838 [08:59<00:00, 31.20it/s]


Epochs: 2 | Loss:  0.135 | Accuracy:  0.754


100%|██████████████████████████████████████████████████████████████████████████████████████████| 16838/16838 [08:31<00:00, 32.91it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████████████| 16838/16838 [08:07<00:00, 34.53it/s]


Epochs: 11 | Loss:  0.094 | Accuracy:  0.886


100%|██████████████████████████████████████████████████████████████████████████████████████████| 16838/16838 [08:14<00:00, 34.05it/s]


Epochs: 12 | Loss:  0.093 | Accuracy:  0.885


 90%|█████████████████████████████████████████████████████████████████████████████████▏        | 15181/16838 [07:16<00:41, 39.80it/s]

# Model Prediction

In [ ]:
def predict(text):

  sequence = torch.tensor([vocab[word] for word in tokenizer(text)], dtype=torch.long).unsqueeze(0)
  output = model(sequence.to(device))
  prediction = id2label[output.argmax(axis=1).item()]

  return prediction

In [ ]:
idx = 24
text = df_test['Message'].values.tolist()[idx]
gt = df_test['Category'].values.tolist()[idx]
prediction = predict(text)

print(f'Text: {text}')
print(f'Ground Truth: {gt}')
print(f'Prediction: {prediction}')

In [ ]:
idx = 35
text = df_test['Message'].values.tolist()[idx]
gt = df_test['Category'].values.tolist()[idx]
prediction = predict(text)

print(f'Text: {text}')
print(f'Ground Truth: {gt}')
print(f'Prediction: {prediction}')